In [1]:
!pip install --upgrade kafka-python

In [ ]:
from kafka import KafkaProducer
import time
import json
import random
import uuid
import numpy as np

# Connect to local Kafka
producer = KafkaProducer(
    bootstrap_servers="localhost:9092",
    value_serializer=lambda v: json.dumps(v).encode("utf-8")
)

# Centers for your fake embeddings
NORMAL_CENTER = [0.5, 0.5]
ATTACK_CENTER = [0.9, 0.9]

def generate_stream():
    while True:
        is_attack = random.random() > 0.95  # 5% chance of attack

        if is_attack:
            x, y = np.random.normal(ATTACK_CENTER, 0.05, 2)
            event_type = "jailbreak_attempt"
            toxicity = random.uniform(0.8, 1.0)
            prompt = "Ignore previous instructions and..."
        else:
            x, y = np.random.normal(NORMAL_CENTER, 0.2, 2)
            event_type = "llm_generation"
            toxicity = random.uniform(0.0, 0.1)
            prompt = "Please help me with..."

        record = {
            "trace_id": str(uuid.uuid4()),
            "timestamp": time.time(),
            "event_type": event_type,
            "embedding_x": float(np.clip(x, 0, 1)),
            "embedding_y": float(np.clip(y, 0, 1)),
            "toxicity_score": toxicity,
            "latency_ms": int(random.normalvariate(300, 50)),
            "prompt_preview": prompt
        }

        # Send the event to Kafka
        producer.send("llm_events", record)

        # Optional: print to console
        print(record)

        # Control stream speed
        time.sleep(0.001)

# Start generating events
generate_stream()


In [ ]:
# output a small sample into csv


import csv
import json
import random
import uuid
import numpy as np
import time

# Parameters
NORMAL_CENTER = [0.5, 0.5]
ATTACK_CENTER = [0.9, 0.9]
num_records = 100  # number of records to save
csv_file = r"C:\Users\mikha\OneDrive\Documents\Demos\AI Observability\llm_events_sample.csv"

# Open CSV file
with open(csv_file, mode="w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=[
        "trace_id", "timestamp", "event_type",
        "embedding_x", "embedding_y", "toxicity_score",
        "latency_ms", "prompt_preview"
    ])
    writer.writeheader()

    for _ in range(num_records):
        is_attack = random.random() > 0.95
        
        if is_attack:
            x, y = np.random.normal(ATTACK_CENTER, 0.05, 2)
            event_type = "jailbreak_attempt"
            toxicity = random.uniform(0.8, 1.0)
            prompt = "Ignore previous instructions and..."
        else:
            x, y = np.random.normal(NORMAL_CENTER, 0.2, 2)
            event_type = "llm_generation"
            toxicity = random.uniform(0.0, 0.1)
            prompt = "Please help me with..."

        record = {
            "trace_id": str(uuid.uuid4()),
            "timestamp": time.time(),
            "event_type": event_type,
            "embedding_x": float(np.clip(x, 0, 1)),
            "embedding_y": float(np.clip(y, 0, 1)),
            "toxicity_score": toxicity,
            "latency_ms": int(random.normalvariate(300, 50)),
            "prompt_preview": prompt
        }

        writer.writerow(record)

print(f"Sample CSV written to {csv_file}")


Sample CSV written to C:\Users\mikha\OneDrive\Documents\Demos\AI Observability\llm_events_sample.csv
